In [1]:
import pandas as pd
import numpy as np

In [2]:
numeric = "../input_orig/train_numeric.csv"
date = "../input_orig/train_date.csv"
categorical = "../input_orig/train_categorical.csv"

numeric_te = "../input_orig/test_numeric.csv"
date_te = "../input_orig/test_date.csv"
categorical_te = "../input_orig/test_categorical.csv"

In [3]:
def orgainize(features):
    line_features = {}
    station_features = {}
    lines = set([f.split('_')[0] for f in features])
    stations = set([f.split('_')[1] for f in features])
    
    for l in lines:
        line_features[l] = [f for f in features if l+'_' in f]
        
    for s in stations:
        station_features[s] = [f for f in features if s+'_' in f]
            
    return line_features, station_features

In [4]:
features_num = pd.read_csv(numeric, nrows=1).drop(['Response', 'Id'], axis=1).columns.values
line_features_num, station_features_num = orgainize(features_num)
print("Features in Station 32: {}".format( station_features_num['S32'] ))

Features in Station 32: ['L3_S32_F3850']


In [5]:
features_date = pd.read_csv(date, nrows=1).drop(['Id'], axis=1).columns.values
line_features_date, station_features_date = orgainize(features_date)
print("Features in Station 32: {}".format( station_features_date['S32'] ))

Features in Station 32: ['L3_S32_D3852']


In [6]:
features_cat = pd.read_csv(categorical, nrows=1).drop(['Id'], axis=1).columns.values
line_features_cat, station_features_cat = orgainize(features_cat)
print("Features in Station 32: {}".format( station_features_cat['S32'] ))

Features in Station 32: ['L3_S32_F3851', 'L3_S32_F3853', 'L3_S32_F3854']


In [7]:
CHUNKSIZE = 200000
NROWS_TR = 1183747
nrows_tr = 0
NROWS_TE = 1183748
nrows_te = 0
ID_COLUMN = 'Id'
TARGET_COLUMN = 'Response'
SEED = 0

tr_sum_full_num = pd.DataFrame()
for tr in pd.read_csv(numeric, chunksize=CHUNKSIZE, dtype=np.float32):
    tmp1 = pd.DataFrame()
    feats = np.setdiff1d(tr.columns, [ID_COLUMN, TARGET_COLUMN])
    tmp1['sum_full_num'] = tr[feats].sum(axis=1)
    tr_sum_full_num = tr_sum_full_num.append(tmp1)

    nrows_tr += CHUNKSIZE
    if nrows_tr >= NROWS_TR:
        break
        
te_sum_full_num = pd.DataFrame()
for te in pd.read_csv(numeric_te, chunksize=CHUNKSIZE, dtype=np.float32):
    tmp2 = pd.DataFrame()
    feats = np.setdiff1d(te.columns, [ID_COLUMN, TARGET_COLUMN])
    tmp2['sum_full_num'] = te[feats].sum(axis=1)
    te_sum_full_num = te_sum_full_num.append(tmp2)
    
    nrows_te += CHUNKSIZE
    if nrows_te >= NROWS_TE:
        break

In [11]:
CHUNKSIZE = 200000
NROWS_TR = 1183747
nrows_tr = 0
NROWS_TE = 1183748
nrows_te = 0
ID_COLUMN = 'Id'
TARGET_COLUMN = 'Response'
SEED = 0

tr_sum_full_date = pd.DataFrame()
for tr in pd.read_csv(date, chunksize=CHUNKSIZE, dtype=np.float32):
    tmp1 = pd.DataFrame()
    feats = np.setdiff1d(tr.columns, [ID_COLUMN])
    tmp1['sum_full_date'] = tr[feats].sum(axis=1)
    tr_sum_full_date = tr_sum_full_date.append(tmp1)

    nrows_tr += CHUNKSIZE
    if nrows_tr >= NROWS_TR:
        break
        
te_sum_full_date = pd.DataFrame()
for te in pd.read_csv(date_te, chunksize=CHUNKSIZE, dtype=np.float32):
    tmp2 = pd.DataFrame()
    feats = np.setdiff1d(te.columns, [ID_COLUMN])
    tmp2['sum_full_date'] = te[feats].sum(axis=1)
    te_sum_full_date = te_sum_full_date.append(tmp2)
    
    nrows_te += CHUNKSIZE
    if nrows_te >= NROWS_TE:
        break

In [16]:
tr_sum_full_date.shape, te_sum_full_date.shape

((1183747, 1), (1183748, 1))

In [12]:
tr_sum_full = pd.concat([tr_sum_full_num,tr_sum_full_date],axis=1)
te_sum_full = pd.concat([te_sum_full_num,te_sum_full_date],axis=1)

In [17]:
tr_sum_full.head()

,sum_full_num,sum_full_date
0,-3.215,15385.011719
1,0.451,274861.000000
2,-0.130,335984.812500
3,-0.123,238673.750000
4,-1.453,125285.210938


In [18]:
te_sum_full.head()

,sum_full_num,sum_full_date
0,-1.366,196310.265625
1,-3.401,274750.187500
2,3.215,174394.093750
3,-1.776,51450.808594
4,1.463,189861.421875


In [13]:
tr_sum_full.to_hdf('tr_sum_full5.hdf5','table')
te_sum_full.to_hdf('te_sum_full5.hdf5','table')